In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from glob import glob
import os
import numpy as np
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
import math
import matplotlib.gridspec as gridspec

# figure save option
save_option = True
figsavepath = "../Figures/FigureS23_TRPL.svg"

# Data path
path = "../Data/FigureS23_TRPL/240419_for-revision.xlsx"

def tri_exponential_decay(t,A1,A2,A3,tau1,tau2,tau3, c):
    return A1 * np.exp((-1)*t/tau1) + A2 * np.exp((-1)*t/tau2) + A3 * np.exp((-1)*t/tau3) + c

def process_data(X,Y,normalize=True):
    # find on-time
    on_index = Y.index(max(Y))
    offset_time = X[on_index]
    X = [x-offset_time for x in X]
    if normalize:
        # normalize
        Y = [(y-min(Y))/max(Y) for y in Y]
    return X, Y, on_index

def TRPL_fit(X, Y, label):
    # Make On time to be 0 ns and Normalize Y
    X, Y, on_index = process_data(X, Y, normalize=True)

    # Cut time for fitting
    power = -3
    threshold = 10 ** power
    while True:
        try:
            cut_index = next(y[0] for y in enumerate(Y[on_index:]) if y[1] < threshold) + on_index
            break
        except:
            power += 0.2
            threshold = 10 ** power
    cut_time = X[cut_index]

    # cut_time = 700
    # cut_index = next(x[0] for x in enumerate(X) if x[1] < cut_time)
    # print(f'{cut_time:.1f} ns')

    # Data used for fitting
    X_for_fit = X[on_index:cut_index]
    Y_for_fit = Y[on_index:cut_index]
    # Prepare log for fitting
    log_Y_for_fit = np.log(Y_for_fit)

    # start from tri exponential
    def log_fit(t, A1, A2, A3, tau1, tau2, tau3, c):
        return np.log(tri_exponential_decay(t, A1, A2, A3, tau1, tau2, tau3, c))
    
    # Initial guess for the parameters
    initial_guess = [0.7, 0.2, 0.1, 5, 50, 100, 0]
    # Bounds for the parameters (all parameters are positive)
    bounds = (0, np.inf)  # All parameters must be greater than 0

    # Fit the tri-exponential function to the data with bounds
    popt, _ = curve_fit(log_fit, X_for_fit, log_Y_for_fit, p0=initial_guess, bounds=bounds)

    # Calculate R-squared value
    Y_fit = [np.exp(log_fit(x, popt[0],popt[1],popt[2],popt[3],popt[4],popt[5], popt[6])) for x in X_for_fit]
    r2 = r2_score(Y_for_fit, Y_fit)
    r2_log = r2_score(log_Y_for_fit, np.log(Y_fit))

    results = list(popt)

    # print('Fit: a1=%5.3f, a2=%5.3f, a3=%5.3f, t1=%5.3f, t2=%5.3f, t3=%5.3f, c=%5.3f' % tuple(popt))
    # print(f'R2: {r2:.4f}, R2_log: {r2_log:.4f}')
    
    # plt.scatter(X, Y, c='white', ec='#4591AA80')
    # plt.plot(X_for_fit, Y_fit, c='#4591AA',label=label)
    # plt.ylabel('Counts')
    # plt.xlabel('Time (ns)')
    # plt.ylim(1e-4, 1)
    # plt.xlim(0, None)
    # plt.yscale('log')
    # plt.show()

    # round values
    cut_time = round(cut_time,1)
    results1 = [round(n,4) for n in results[:3]] # A1, A2, A3
    results2 = [round(n,2) for n in results[3:6]] # t1, t2, t3
    results3 = [round(results[-1],4)] # c
    results = results1 + results2 + results3
    r2 = round(r2,4)
    r2_log = round(r2_log,4)
    return X, Y, cut_time, results, r2, r2_log

# Data reading and fitting
df = pd.read_excel(path,skiprows=1)
data_list = []
for i in range(12):
    X = df.iloc[:,0+(i*2)].to_list()
    Y = df.iloc[:,1+(i*2)].to_list()
    X = [x for x in X if not math.isnan(x)]
    Y = [y for y in Y if not math.isnan(y)]
    data_list.append([X,Y])

result_list = []
for data in data_list:
    X, Y = data[0], data[1]
    X, Y, cut_time, results, r2, r2_log = TRPL_fit(X, Y, label='')
    result_list.append([X, Y, cut_time, results, r2, r2_log])

# Plot
plt.rcParams["font.size"] = 17
fig = plt.figure(figsize=(14.5, 5))
gs = gridspec.GridSpec(1, 2, width_ratios=[5,5])

# Axes1
ax1 = plt.subplot(gs[0])
# Plot representative
plot_ids = [1,4,6,9]
#HG-1: 230925_B, HG-2: 240228_A, LG-1: 240304_AY1, LG-2: 240304_AY2
x_labels = ['HG-1#1','HG-1#2','HG-1#3','HG-2#1','HG-2#2','HG-2#3','LG-1#1','LG-1#2','LG-1#3','LG-2#1','LG-2#2','LG-2#3']
sample_colors = ['#01ADC1', '#4591AA', '#8A7594', '#CF597E']

for i in plot_ids:
    X = result_list[i][0]
    Y = result_list[i][1]
    a1, a2, a3, t1, t2, t3, c = result_list[i][3]
    Y_fit = [tri_exponential_decay(x,a1,a2, a3, t1, t2, t3, c) for x in X]
    ax1.scatter(X,Y,ec=f'{sample_colors[i//3]}80',c='white',s=30)
    ax1.plot(X,Y_fit,label=f'{x_labels[i][:-2]}: {t3:.1f} ns',color=sample_colors[i//3])

ax1.set_xlabel('Time (ns)')
ax1.set_ylabel('Counts (a.u.)')
ax1.set_yscale('log')
ax1.set_xlim(-50,900)
ax1.set_ylim(1e-3,1)
ax1.legend(frameon=False)


# Axes 2
ax2 = plt.subplot(gs[1])
y_values = []
bar_colors = []
for i in range(12):
    y_values.append(result_list[i][3][-2])
    bar_colors.append(sample_colors[i//3])
ax2.bar(x_labels,y_values,color=bar_colors)
ax2.set_ylabel('Lifetime (ns)')
ax2.set_xticklabels(x_labels, rotation=45) 

# Show and save the plot
plt.tight_layout()
if save_option:
    plt.savefig(figsavepath, dpi=1200, bbox_inches='tight', transparent=True)
plt.show()


In [ ]:
# average lifetimes
mean = 0
count = 0
for y in y_values:
    mean += y
    count += 1
    if count%3 == 0:
        print(np.mean(mean)/3)
        mean = 0